In [1]:
from scipy.sparse import csc_matrix
import trimesh

mesh = trimesh.load("/Users/ackermand/Documents/Downloads/410_roi1.obj")



In [2]:
import networkx as nx
g = nx.from_edgelist(mesh.edges_unique)

In [20]:
import numpy as np
import time
metric = np.random.random(len(mesh.vertices))
t = time.time()
face_metrics = np.mean(metric[mesh.faces.flatten()].reshape((-1,3)),axis=1)
print(len(face_metrics),shape(mesh.faces)

1209490


In [5]:

import numpy as np
import time
e = np.array(mesh.edges)
t = time.time()
temp = csc_matrix((np.ones(len(mesh.faces)),(e[:,0],e[:,1])), shape=(len(mesh.vertices),len(mesh.faces)))
print(time.time()-t)
t = time.time()
# for i in range(2):
#     temp = temp.dot(temp)
print(type(temp))
print(time.time()-t)
rows,cols = temp.nonzero()
cols[rows==0]

ValueError: row, column, and data array must all be the same length

In [47]:
g = nx.from_edgelist(mesh.edges_unique)

In [9]:
a = dict(nx.all_pairs_shortest_path_length(g,cutoff=5))

KeyboardInterrupt: 

In [8]:
a[0][0]

TypeError: 'generator' object is not subscriptable

In [1]:
import pyglet
window = pyglet.window.Window()
label = pyglet.text.Label('Hello, world',
                          font_name='Times New Roman',
                          font_size=36,
                          x=window.width//2, y=window.height//2,
                          anchor_x='center', anchor_y='center')

@window.event
def on_draw():
    window.clear()
    label.draw()

pyglet.app.run()

2022-09-01 13:36:32.544 python[65827:7455854] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7f89d3168380>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-09-01 13:36:32.545 python[65827:7455854] Warning: Expected min height of view: (<NSButton: 0x7f89d30dbc60>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-09-01 13:36:32.547 python[65827:7455854] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7f89d30de630>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2022-09-01 13:36:32.549 python[65827:7455854] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7f89d30e7a40>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


: 

In [5]:
import numpy as np
a = np.arange(10)
a.clip(4,7)
print(a)

[0 1 2 3 4 5 6 7 8 9]
